In [1]:
import tensorflow as tf
os.getcwd()

'c:\\Users\\arunk\\Deep Learning\\DEEPCNNClassifier\\research'

In [37]:
import os
# os.chdir(("../"))

In [60]:

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/ArunKhare/DEEPCNNClassifier.mlflow" 
os.environ["MLFLOW_TRACKING_USERNAME"]="ArunKhare"
os.environ["MLFLOW_TRACKING_PASSWORD"]="73344eca5240dfe224b69018f8be21b9fd8df6c4"

In [61]:
model =tf.keras.Model("artifacts/training/model.h5")
model

In [62]:
from urllib.parse import urlparse
import mlflow
import mlflow.keras

In [63]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [64]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/PetImages",
            mlflow_uri="https://dagshub.com/ArunKhare/DEEPCNNClassifier.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
      


In [67]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [68]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()
    
except Exception as e:
   raise e

(2022-10-02 11:23:05,822:INFO:common: yaml file: configs\config.yaml loaded successfully)
(2022-10-02 11:23:05,825:INFO:common: yaml file: params.yaml loaded successfully)
(2022-10-02 11:23:05,827:INFO:common: created directory at: artifacts)
Found 7498 images belonging to 2 classes.
469/469 [==============================] - 48s 86ms/step - loss: 8.0334 - accuracy: 0.7665
(2022-10-02 11:23:56,231:INFO:common: json file saved at: scores.json)
(2022-10-02 11:24:11,205:WARNING:save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.)
INFO:tensorflow:Assets written to: C:\Users\arunk\AppData\Local\Temp\tmpe1vt3f4z\model\data\model\assets
(2022-10-02 11:24:11,989:INFO:builder_impl: Assets written to: C:\Users\arunk\AppData\Local\Temp\tmpe1vt3f4z\model\data\model\assets)

c:\Users\arunk\ComputerVision\CNN\DEEPCNNClassifier\env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2022/10/02 11:24:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
